# Zaawansowany Python

## lab 2 - Wybrane zagadnienia programowania obiektowego w Pythonie

plan:

* dataclass - tutaj dałbym jeszcze collections.namedtuple i typing.NamedTuple
* wypisanie jakichś zalet i wad oraz może porównanie prędkości dla podobnych funkcjonalności z wykorzystanie każdego z elementów?
* abc classes ? czy tu
* dziedziczenie - szczególnie multidziedziczenie czy tu

## 1. collections.namedtuple, typing.NamedTuple oraz Dataclass

Dokumentacja:

* collections.namedtuple:
  * https://docs.python.org/3/library/collections.html#collections.namedtuple
* typing.NamedTuple:
  * https://docs.python.org/3/library/typing.html#typing.NamedTuple
* Dataclass:
  * https://peps.python.org/pep-0557/
  * https://docs.python.org/3/library/dataclasses.html


### 1.1 collections.namedtuple

Klasa `namedtuple` umożliwia stworzenie obiektu krotko-podobnego umożliwiającego donanie nazw do wartości, które możemy w ramach tego obiektu przechować. Zachowujemy wiele cech krotek, ale dodajemy również możliwości definicji cech jak przy tworzeniu własnych klas.

**Listing 1**

In [2]:
from collections import namedtuple

# definicja krotki nazwanej
User = namedtuple('User', ['firstname','lastname','age','email'])

# inicjalizacja obiektu krotki nazwanej
u = User('Jan','Nowak',34,'jan.nowak@student.uwm.edu.pl')
print(u)

# jak to z krotkami - są niemutowalne (niezmienne)
u.firstname = 'Adam'

User(firstname='Jan', lastname='Nowak', age=34, email='jan.nowak@student.uwm.edu.pl')


AttributeError: can't set attribute

In [43]:
# dodajmy jeszcze jedną krotkę na potrzeby przykładu z wczytaniem danych z pliku

Order = namedtuple('Order', 'Kraj, Sprzedawca, Data_zamowienia, idZamowienia, Utarg')

Krotki nazwane mogą nam się przydać np. przy wczytywaniu danych z plików czy baz danych, gdzie chcemy na pewnym etapie mieć obiekt, którego nie można tak łatwo (celowo lub przypadkiem) zmodyfikować.

In [44]:
# plik z danymi jest dołączony do notebooka
import csv

# _make jest odpowiedzialne za utworzenie instancji obiektu krotki z przekazanych argumentów
orders = map(Order._make, csv.reader(open("zamowienia.csv", encoding='utf8'), delimiter=';'))
orders = list(orders)

for order in orders[:5]:
    print(order.Sprzedawca, order.Utarg, type(order))

Sprzedawca Utarg <class '__main__.Order'>
Kowalski 440.00 <class '__main__.Order'>
Sowiński 1863.40 <class '__main__.Order'>
Peacock 1552.60 <class '__main__.Order'>
Leverling 654.06 <class '__main__.Order'>


In [45]:
# kilka wybranych metod i właściwości krotki nazwanej
orders[1]._asdict()

print(orders[1]._replace(Sprzedawca = 'Nowak'))
# ale nie podmienia tego w trybie in place
orders[1]

Order(Kraj='Polska', Sprzedawca='Nowak', Data_zamowienia='2003-07-16', idZamowienia='10248', Utarg='440.00')


Order(Kraj='Polska', Sprzedawca='Kowalski', Data_zamowienia='2003-07-16', idZamowienia='10248', Utarg='440.00')

In [46]:
print(orders[1]._fields)

('Kraj', 'Sprzedawca', 'Data_zamowienia', 'idZamowienia', 'Utarg')


In [47]:
# istnieje również możliwość zdefiniowania wartości domyślnej, nieco w ograniczony sposób
Point = namedtuple('Point', ['x','y'], defaults=[0,0])
p = Point()
p._asdict()

{'x': 0, 'y': 0}

In [33]:
p1 = Point(2,2)

In [39]:
# klasyczne krotki nazwane dostarczają również domyślnej implementacji
# metod zaimplementowanych w bazowej klasie tuple
print(p > p1, 0 in p, len(p1) == 2)

False True True


### 1.2 typing.NamedTuple

Typowane krotki nazwane są slternatywną wersją dla klasycznych krotek nazwanych pozwalającą na zdefiniowanie wskazówek typu.

In [42]:
# dla ułatwienia nazwy pól zostały zamienione na bardziej odpowiednie dla Pythona
from typing import NamedTuple


class NewOrder(NamedTuple):
    kraj: str
    sprzedawca: str
    data_zamowienia: str
    id_zamowienia: int
    utarg: float

# dla ułatwiena introspekcji obiektów typu NamedTuple wsprowadzono dodatkowy atrybut
NewOrder.__annotations__

{'kraj': str,
 'sprzedawca': str,
 'data_zamowienia': str,
 'id_zamowienia': int,
 'utarg': float}

In [60]:
# mimo iż wskazówki typów są określone, to nie otrzymujemy żadnego ostrzeżenia jeżeli
# przypisujemy wartości innych typów
new_order_1 = NewOrder(*orders[1]._asdict().values())
new_order_2 = NewOrder(*orders[2]._asdict().values())
new_order_1, new_order_2

(NewOrder(kraj='Polska', sprzedawca='Kowalski', data_zamowienia='2003-07-16', id_zamowienia='10248', utarg='440.00'),
 NewOrder(kraj='Polska', sprzedawca='Sowiński', data_zamowienia='2003-07-10', id_zamowienia='10249', utarg='1863.40'))

In [63]:
print(new_order_1 == new_order_2, new_order_1 < new_order_2)

False True


Szczegółowe wyjaśnienie zasad porównywania krotek można znaleźć np.tu:

https://www.askpython.com/python/tuple/tuple-comparison

### 1.2 Dataclass - klasy danych

> Dokumentacja: https://docs.python.org/3/library/dataclasses.html  
> Specyfikacja: https://peps.python.org/pep-0557/

Data class, która najczęściej używana jest jako dekorator, dostarcza rozwiązania pozwalającego stworzyć coś na wzór pojemnika na dane (struktury) z dodatkową możliwością automatycznego wygenerowania implementacji niektórych metod magicznych np. porównujących wartości atrybutów w tych klasach. Dataclass przechowuje właściwości w postaci pól (ang. `fields`), które mogą być typowane, ale poza pewnymi wyjątkami nie są one restrykcyjnie sprawdzane.

Sygnatura dekoratora wygląda jak poniżej:  
`def dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False,
           match_args=True, kw_only=False, slots=False, weakref_slot=False)`

Atrybuty nazwane określają czy generowane ma być wskazana metoda. W przypadku init, repr oraz eq sytuacja jest dość czywista jeżeli chodzi o nazwy tych metod, ale warto wskazać, że dla agumentu order chodzi o metody  `__lt__()`, `__le__()`, `__gt__()`, and `__ge__()`.
Atrybut `unsafe_hash` określa czy generowana będzie metoda `__hash__()`, której powstanie jest uzależnione od wartości atrybutów `eq` oraz `frozen` a cała logika jest dość mocno zawiła, więc odsyłam do dokumentacji podanej wcześniej. Atrybut `frozen` określa czy możliwe jest przypisywanie wartości do atrybutów po inicjalizacji obiektu, więc można stworzyś coś na wzór obiektu niemutowalnego.

In [104]:
from dataclasses import dataclass, field, make_dataclass

In [80]:
# klasyczna klasa z adnotacjami
class User:
    username: str # adnotacja
    email: str # adnotacja
    is_active: bool = False # adnotacja z wartością domyślną
    is_admin = False # pole na poziomie klasy (nie instancji)

u = User()
print(u.__dict__)
print(u.__annotations__)
print(u.is_admin)
print(u.username) # błąd

{}
{'username': <class 'str'>, 'email': <class 'str'>, 'is_active': <class 'bool'>}
False


AttributeError: 'User' object has no attribute 'username'

In [81]:
u.username = 'adept'

In [82]:
u.__dict__

{'username': 'adept'}

In [83]:
# domyślny konstruktor odziedziczony po klasie bazowej jest bezargumentowy
# więc to nie zadziała
u2 = User('zealot', 'zealot@student.uwm.edu.pl')

TypeError: User() takes no arguments

In [85]:
@dataclass
class User:
    username: str # adnotacja
    email: str # adnotacja
    is_active: bool = False # adnotacja z wartością domyślną
    is_admin = False # pole na poziomie klasy (nie instancji)

u = User()

TypeError: User.__init__() missing 2 required positional arguments: 'username' and 'email'

Powyższy błąd wynika z faktu, że domyślnie `init=True` w sygnaturze dekoratora `@dataclass`. Możemy albo zmienić wartość tego atrybutu przy inicjalizacji albo kontruować obiekty podając wymagane argumenty.

In [95]:
u = User('zealot','zealot@student.uwm.edu.pl', True)
print(u) # domyślnie repr=True, więc zobaczymy coś więcej niż domyślny łańcuch znaków dla obiektu

User(username='zealot', email='zealot@student.uwm.edu.pl', is_active=True)


In [96]:
from copy import deepcopy
u1 = deepcopy(u)
u == u1

True

In [98]:
u1.username = 'adept'
print(u1)
print(u)
u == u1

User(username='adept', email='zealot@student.uwm.edu.pl', is_active=True)
User(username='zealot', email='zealot@student.uwm.edu.pl', is_active=True)


False

In [101]:
# pola klasy dataclass można również deklarować z użyciem jawnego wywołania dataclass.field
# sygnatura: dataclasses.field(*, default=MISSING, default_factory=MISSING, init=True, repr=True, 
# hash=None, compare=True, metadata=None, kw_only=MISSING)
# więcej: https://docs.python.org/3/library/dataclasses.html#dataclasses.field
@dataclass
class User:
    username: str # adnotacja
    email: str = field(repr=False)
    is_active: bool = False # adnotacja z wartością domyślną
    is_admin = False # pole na poziomie klasy (nie instancji)

In [102]:
u = User('zealot','zealot@student.uwm.edu.pl', True)
print(u)

User(username='zealot', is_active=True)


Możliwe jest również dynamiczne tworzenie definicji klasy danych. Przykład z oficjalnej dokumentacji poniżej.

In [105]:
C = make_dataclass('C',
                   [('x', int),
                     'y',
                    ('z', int, field(default=5))],
                   namespace={'add_one': lambda self: self.x + 1})

In [106]:
# ekwiwalent w postaci deklaracji klasy
@dataclass
class C:
    x: int
    y: 'typing.Any'
    z: int = 5

    def add_one(self):
        return self.x + 1

**Zadania**



**Zadanie 1**  
Wykorzystując przykłady stwórz funkcję, która stworzy (i zwróci) klasyczną krotkę nazwaną o nazwie `Order`, ale pola w niej zdefiniowane będą dynamiczne, na podstawie wartości z pierwszego wiersza pliku zamowienia.csv. Te nazwy powinny być zamienione na małe znaki i spacje zamienione na znak `_`. Wczytaj dane z pliku opakowując każdy wiersz w tę krotkę i wyświetl pierwsze 10 wierszy (pomijając nagłówkowy).

**Zadanie 2**  
Wykorzystaj krotkę `Point` z przykładów a następnie napisz kod, który przetestuje wszystkie metody porównywania krotek (sprawdź, które metody magiczne są przeciążone), wybrane operacje arytmetyczne (o ile są możliwe).

**Zadanie 3**  
Mając do dyspozycji słownik poniższej postaci stwórz funkcję, króta odczyta dane z tego słownika i dynamicznie (za pomocą make_dataclass) będzie tworzyło klasy danych na jego podstawie.

```python
slownik = {
    1 : {
        'class_name': 'Osoba', 
        'props': [('name', 'str'), ('is_admin', 'bool', 'False')]},
    2 : {
        'class_name': 'Produkt', 
        'props': [('name', 'str'), ('price', 'float', '0.0'), ('quantity', 'float', '0.0')]}
    }
```